# Demo: Reading a short-ranged prediction
- The national water model outputs many .nc files in their AWS bucket
- This tutorial is meant to show how to batch speficic forecasts and variables from many NC files in one xarray object rather than reading everything to disk one by one

![The example of the NWM Bucket](../docs/img/bucket.png)
This is an example of the bucket and how there are 1,700 .nc files generated for *just* the short ranged forecast @ https://noaa-nwm-pds.s3.amazonaws.com/index.html

In [4]:
import nwm_batcher

In [5]:
nwm_batcher.read(
    date="20250216",
    forecast_type="short_range",
    initial_time="t00z",
    variable="channel_rt"
)

<xarray.Dataset> Size: 2GB
Dimensions:         (time: 18, feature_id: 2776734, reference_time: 1)
Coordinates:
    time            (time) int32 72B cubed.Array<chunksize=(1,)>
    reference_time  (reference_time) int32 4B cubed.Array<chunksize=(1,)>
    feature_id      (feature_id) int64 22MB cubed.Array<chunksize=(2776734,)>
Data variables:
    nudge           (time, feature_id) float64 400MB cubed.Array<chunksize=(1, 2776734)>
    velocity        (time, feature_id) float64 400MB cubed.Array<chunksize=(1, 2776734)>
    qSfcLatRunoff   (time, feature_id) float64 400MB cubed.Array<chunksize=(1, 2776734)>
    qBucket         (time, feature_id) float64 400MB cubed.Array<chunksize=(1, 2776734)>
    qBtmVertRunoff  (time, feature_id) float64 400MB cubed.Array<chunksize=(1, 2776734)>
    streamflow      (time, feature_id) float64 400MB cubed.Array<chunksize=(1, 2776734)>